In [1]:
import json
import os
from datetime import datetime

In [2]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: :/Users/carlos/WORKSPACE/MegaClassifier/data/external/ai4eutils:/Users/carlos/WORKSPACE/MegaClassifier/data/external/CameraTraps:/Users/carlos/WORKSPACE/MegaClassifier/data/external/yolov5


In [3]:
threshold = float(0.01)

OUTPUT_PATH = os.path.abspath("./resources/json")
MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5a.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5b.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v4.1.0.pb")

file_name = "prueba.json"
OUTPUT_FILE_PATH = os.path.join(OUTPUT_PATH, file_name)

os.makedirs(OUTPUT_PATH, exist_ok=True)

print("MODEL_PATH:", MODEL_PATH)
print("OUTPUT_FILE_PATH:", OUTPUT_FILE_PATH)
print("OUTPUT_PATH:", OUTPUT_PATH)

MODEL_PATH: /Users/carlos/WORKSPACE/MegaClassifier/models/MegaDetector/md_v5a.0.0.pt
OUTPUT_FILE_PATH: /Users/carlos/WORKSPACE/MegaClassifier/resources/json/prueba.json
OUTPUT_PATH: /Users/carlos/WORKSPACE/MegaClassifier/resources/json


In [4]:
detection_inicialitation_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

In [5]:
!python data/external/CameraTraps/detection/run_detector_batch.py \
    "$MODEL_PATH" \
    /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP \
    "$OUTPUT_FILE_PATH" \
    --recursive \
    --threshold "$threshold"

60 image files found in the input directory
PyTorch reports 0 available CUDA devices
GPU available: False
Using PyTorch version 1.10.1
Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients
Loaded model in 1.67 seconds
Loaded model in 1.67 seconds
100%|███████████████████████████████████████████| 60/60 [02:43<00:00,  2.73s/it]
Finished inference for 60 images in 2 minutes and 46.06 seconds
Output file saved at /Users/carlos/WORKSPACE/MegaClassifier/resources/json/prueba.json
Done!


In [7]:
with open(OUTPUT_FILE_PATH, "r") as file:
    data = json.load(file)

for image in data["images"]:
    image["file"] = (
        image["file"]
        .replace(
            "/Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP",
            "",
        )
        .replace("/", "\\")
    )

info = {
    "detection_initial_time": detection_inicialitation_time,
    "detection_completion_time": data["info"]["detection_completion_time"],
    "format_version": data["info"]["format_version"],
    "detector": data["info"]["detector"],
    "detector_threshold": threshold,
    "detector_metadata": data["info"]["detector_metadata"],
}

final_output = {
    "images": data["images"],
    "detection_categories": data["detection_categories"],
    "info": info,
}

threshold_str = str(threshold).replace(".", "_")
json_name = f"{len(data['images'])}_images_{threshold_str}_threshold.json"
model_name = os.path.basename(MODEL_PATH).split(".")[0]
tmp_path = os.path.join(OUTPUT_PATH, model_name)
os.makedirs(tmp_path, exist_ok=True)
NEW_OUTPUT_FILE_PATH = os.path.join(tmp_path, json_name)


with open(NEW_OUTPUT_FILE_PATH, "w") as f:
    json.dump(final_output, f, indent=1)
print(f"Output file saved at {NEW_OUTPUT_FILE_PATH}")

os.remove(OUTPUT_FILE_PATH)
print(f"Output file deleted: {OUTPUT_FILE_PATH}")


Output file saved at /Users/carlos/WORKSPACE/MegaClassifier/resources/json/md_v5a/60_images_0_01_threshold.json
Output file deleted: /Users/carlos/WORKSPACE/MegaClassifier/resources/json/prueba.json
